# Circuit Breaker

##  STEP #1: Notebook setup

In [39]:
pwd

/Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/circuit-breaker-hystrix


In [4]:
git clone "https://github.com/svergara1987/tesis-maestria-eventb-compliance.git"

Cloning into 'tesis-maestria-eventb-compliance'...
remote: Enumerating objects: 1120, done.
remote: Counting objects: 100% (946/946), done.
remote: Compressing objects: 100% (461/461), done.
remote: Total 1120 (delta 352), reused 817 (delta 229), pack-reused 174
Receiving objects: 100% (1120/1120), 127.99 MiB | 7.15 MiB/s, done.
Resolving deltas: 100% (374/374), done.
Updating files: 100% (321/321), done.


##  STEP #2: Generate abstract test cases from the Event-B machine

Set JAVA_HOME to Java 8

In [69]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/jdk1.8.0_261.jdk/Contents/Home"

In [70]:
java -version

java version "1.8.0_261"
Java(TM) SE Runtime Environment (build 1.8.0_261-b12)
Java HotSpot(TM) 64-Bit Server VM (build 25.261-b12, mixed mode)


Show `eventb2xml-circuit-breaker-strategy.yaml` file contents

In [49]:
cat "test/eventb2xml-circuit-breaker-strategy.yaml"

---
apiVersion: 1
kind: cbc
metadata:
  name: nodes coverage
expression:
  - circuit_breaker=CLOSED
  - circuit_breaker=HALF_OPEN
  - circuit_breaker=OPEN
steps: 100
timeout: 60 #in seconds

---
apiVersion: 1
kind: sequence
metadata:
  name: edges coverage
events:
  - request(microservice_response=FALSE)
  - request(microservice_response=FALSE)
  - request(microservice_response=FALSE)
  - clock
  - request(microservice_response=FALSE)

---
apiVersion: 1
kind: random
metadata:
  name: random generation
count: 3
steps: 30
timeout: 60

Delete any previously generated file with this notebook

In [75]:
rm -rf "test/cb.xml"
rm -rf "test/delete_cb.xml"

Execute the `probcli_test_generator.py` tool

In [76]:
python3 eventb2xml/py/probcli_test_generator.py --strategy "test/eventb2xml-circuit-breaker-strategy.yaml" --machine "test/eventb_machines/m0_circuit_breaker_mch.eventb" --output "test/cb.xml" --keep --verbose

2021-09-12 23:59:26,513 - program arguments=Namespace(keep=True, machine='test/eventb_machines/m0_circuit_breaker_mch.eventb', output='test/cb.xml', strategy='test/eventb2xml-circuit-breaker-strategy.yaml', verbose=True)
2021-09-12 23:59:26,513 - getting absolute paths from relatives
2021-09-12 23:59:26,514 - machine file 'test/eventb_machines/m0_circuit_breaker_mch.eventb' -> '/Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test/eventb_machines/m0_circuit_breaker_mch.eventb'
2021-09-12 23:59:26,514 - test strategy file 'test/eventb2xml-circuit-breaker-strategy.yaml' -> '/Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test/eventb2xml-circuit-breaker-strategy.yaml'
2021-09-12 23:59:26,515 - /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test/eventb2xml-circuit-breaker-strategy.yaml parsed
2021-09-12 23:59:26,518 - document[1]={'apiV

Show `cb.xml` file contents

In [81]:
cat test/cb.xml

<?xml version='1.0' encoding='UTF-8'?>
<extended_test_suite>
	<test_case>
    <initialisation>
      <value name="THRESHOLD" type="constant">3</value>
      <value name="AMOUNT_TEST_REQUESTS" type="constant">3</value>
      <value name="TIMEOUT_PERIOD" type="constant">2</value>
      <value name="test_request_to_go" type="variable">3</value>
      <value name="timestamp_cb_trips" type="variable">0</value>
      <value name="consecutive_errors" type="variable">0</value>
      <value name="circuit_breaker" type="variable">CLOSED</value>
      <value name="time" type="variable">0</value>
    </initialisation>
    <step name="request">
      <value name="microservice_response">FALSE</value>
      <value name="new_circuit_breaker">CLOSED</value>
      <value name="new_consecutive_errors">1</value>
      <value name="new_test_request_to_go">3</value>
      <value name="new_timestamp_cb_trips">0</value>
      <modified name="consecutive_errors">1</modified>
    </step>
    <step name="request

##  STEP #3 (Hystrix): Instantiate abstract test cases

Set JAVA_HOME to Java 16

In [99]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [100]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Package `xml2junit`project

In [101]:
mvn package -f xml2junit/pom.xml

[INFO] Scanning for projects...
[INFO] 
[INFO] -------------------< uy.fing.edu.svergara:xml2junit >-------------------
[INFO] Building xml2junit 0.0.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ xml2junit ---
[WARNING] Using platform encoding (UTF-8 actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] Copying 4 resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ xml2junit ---
[INFO] Changes detected - recompiling the module!
[WARNING] File encoding has not been set, using platform encoding UTF-8, i.e. build is platform dependent!
[INFO] Compiling 19 source files to /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/xml2junit/target/classes
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (default-testResources) @ xml2junit ---
[WARNING] Using platfo

Show `xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml` file contents

In [111]:
cat "test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml"

---
projectLocation: /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test
groupId: uy.edu.fing.svergara
artifactId: junit-circuit-breaker-hystrix-notebook
types:
  - name: CircuitBreaker
    supertype: enum
    values: [CLOSED, OPEN, HALF_OPEN]
variables:
  - name: THRESHOLD
    type: Integer
  - name: AMOUNT_TEST_REQUESTS
    type: Integer
  - name: TIMEOUT_PERIOD
    type: Integer
  - name: test_request_to_go
    type: Integer
  - name: timestamp_cb_trips
    type: Integer
  - name: consecutive_errors
    type: Integer
  - name: circuit_breaker
    type: CircuitBreaker
  - name: microservice_response
    type: Boolean
  - name: new_circuit_breaker
    type: CircuitBreaker
    ignore: true
  - name: new_consecutive_errors
    type: Integer
    ignore: true
  - name: new_test_request_to_go
    type: Integer
    ignore: true
  - name: new_timestamp_cb_trips
    type: Integer
    ignore: true
  - name: consecutive_errors
    type: Integ

Prior executing the following cell, remember to change in `test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml`:
1) The `projectLocation` value in which the junit project will be created
2) The `artifactId` value to a unique name non existing in `projectLocation` folder

In [110]:
java -jar xml2junit/target/xml2junit-0.0.1-SNAPSHOT-jar-with-dependencies.jar -s "test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml" -t "test/cb.xml" --verbose

2021-09-13 22:44:53 INFO uy.fing.edu.svergara.xml2junit.XML2Junit execute yamlTestGenStrategyPath="test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml" and xmlTestCasesPath="test/cb.xml"
2021-09-13 22:44:53 INFO uy.fing.edu.svergara.xml2junit.XML2Junit execute yamlTestGenStrategyPath="test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml" and xmlTestCasesPath="test/cb.xml"
2021-09-13 22:44:53 FINEST uy.fing.edu.svergara.xml2junit.XML2Junit parseYamlTestGenStrategyPath test/xml2junit-junit-circuit-breaker-hystrix-test-gen.yaml
2021-09-13 22:44:54 FINEST uy.fing.edu.svergara.xml2junit.XML2Junit parseYamlTestGenStrategyPath TestGenStrategy [artifactId=junit-circuit-breaker-hystrix-notebook, groupId=uy.edu.fing.svergara, projectLocation=/Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test, types=[Type [name=CircuitBreaker, supertype=enum, values=[CLOSED, OPEN, HALF_OPEN]]], variables=[Variable [ignore=false, name=THRESHOLD, ty

##  STEP #4 (Hystrix): Fill the Wrapper.java contents

In [122]:
cat test/junit-circuit-breaker-hystrix-generated/src/main/java/uy/edu/fing/svergara/Wrapper.java

package uy.edu.fing.svergara;

public class Wrapper {
	
	public void initialisation (Integer AMOUNT_TEST_REQUESTS, Integer THRESHOLD, Integer TIMEOUT_PERIOD, CircuitBreaker circuit_breaker, Integer consecutive_errors, Integer test_request_to_go, Integer time, Integer timestamp_cb_trips) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public Boolean isValid (CircuitBreaker circuit_breaker, Integer consecutive_errors, Integer test_request_to_go, Integer time, Integer timestamp_cb_trips) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public void request (Boolean microservice_response) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public void clock () {
		throw new RuntimeException("code not implemented yet");
	}
		
}


##  STEP #5 (Hystrix): Run circuit breaker implementation

Set JAVA_HOME to Java 16

In [123]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [124]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Package `circuit-breaker-hystrix` project

In [126]:
mvn package -f circuit-breaker-hystrix

[INFO] Scanning for projects...
[INFO] 
[INFO] ----------< uy.edu.fing.uy.svergara:circuit-breaker-hystrix >-----------
[INFO] Building circuit-breaker-hystrix 0.0.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:3.1.0:resources (default-resources) @ circuit-breaker-hystrix ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[INFO] Copying 0 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.1:compile (default-compile) @ circuit-breaker-hystrix ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- maven-resources-plugin:3.1.0:testResources (default-testResources) @ circuit-breaker-hystrix ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/circuit-breaker-hystrix/src/test/resources
[INFO] 
[

The following will definitely not end running as it startup a tomcat server. Run it it in an independant terminal or run it with the detached flag (&) at the end. 

In [129]:
java -jar circuit-breaker-hystrix/target/circuit-breaker-hystrix-0.0.1-SNAPSHOT.jar & 

[1] 29748


##  STEP #6 (Hystrix): Execute concrete test cases against implementation

In [131]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [132]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Run `junit-circuit-breaker-hystrix` project (or the project generated in Step#3 and filled in Step#4)

In [ ]:
mvn package -f test/junit-circuit-breaker-hystrix

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------< uy.edu.fing.svergara:junit-circuit-breaker-hystrix >---------
[INFO] Building junit-circuit-breaker-hystrix 1.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ junit-circuit-breaker-hystrix ---
[WARNING] Using platform encoding (UTF-8 actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] skip non existing resourceDirectory /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/test/junit-circuit-breaker-hystrix/src/main/resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ junit-circuit-breaker-hystrix ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (default-testResources) @ junit-circuit-breaker-hystrix ---
[WARNING] Using platform encoding (UT

##  STEP #3 (Resilience4j): Instantiate abstract test cases

Set JAVA_HOME to Java 16

In [99]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [100]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Package `xml2junit`project

In [101]:
mvn package -f xml2junit/pom.xml

[INFO] Scanning for projects...
[INFO] 
[INFO] -------------------< uy.fing.edu.svergara:xml2junit >-------------------
[INFO] Building xml2junit 0.0.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ xml2junit ---
[WARNING] Using platform encoding (UTF-8 actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] Copying 4 resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ xml2junit ---
[INFO] Changes detected - recompiling the module!
[WARNING] File encoding has not been set, using platform encoding UTF-8, i.e. build is platform dependent!
[INFO] Compiling 19 source files to /Users/svergara/Desktop/tesis-maestria-eventb-compliance/notebooks/tesis-maestria-eventb-compliance/xml2junit/target/classes
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (default-testResources) @ xml2junit ---
[WARNING] Using platfo

Show `xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml` file contents

In [6]:
cat "test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml"

---
projectLocation: /Users/svergara/Desktop/tesis-maestria-eventb-compliance/test
groupId: uy.edu.fing.svergara
artifactId: junit-circuit-breaker-resilience4j-notebook
types:
  - name: CircuitBreaker
    supertype: enum
    values: [CLOSED, OPEN, HALF_OPEN]
variables:
  - name: THRESHOLD
    type: Integer
  - name: AMOUNT_TEST_REQUESTS
    type: Integer
  - name: TIMEOUT_PERIOD
    type: Integer
  - name: test_request_to_go
    type: Integer
  - name: timestamp_cb_trips
    type: Integer
  - name: consecutive_errors
    type: Integer
  - name: circuit_breaker
    type: CircuitBreaker
  - name: microservice_response
    type: Boolean
  - name: new_circuit_breaker
    type: CircuitBreaker
    ignore: true
  - name: new_consecutive_errors
    type: Integer
    ignore: true
  - name: new_test_request_to_go
    type: Integer
    ignore: true
  - name: new_timestamp_cb_trips
    type: Integer
    ignore: true
  - name: consecutive_errors
    type: Integer
  - name: time
    type: Integer


Prior executing the following cell, remember to change in `test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml`:
1) The `projectLocation` value in which the junit project will be created
2) The `artifactId` value to a unique name non existing in `projectLocation` folder

In [8]:
java -jar xml2junit/target/xml2junit-0.0.1-SNAPSHOT-jar-with-dependencies.jar -s "test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml" -t "test/cb.xml" --verbose

2021-09-14 22:42:55 INFO uy.fing.edu.svergara.xml2junit.XML2Junit execute yamlTestGenStrategyPath="test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml" and xmlTestCasesPath="test/cb.xml"
2021-09-14 22:42:55 INFO uy.fing.edu.svergara.xml2junit.XML2Junit execute yamlTestGenStrategyPath="test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml" and xmlTestCasesPath="test/cb.xml"
2021-09-14 22:42:55 FINEST uy.fing.edu.svergara.xml2junit.XML2Junit parseYamlTestGenStrategyPath test/xml2junit-junit-circuit-breaker-resilience4j-test-gen.yaml
2021-09-14 22:42:55 FINEST uy.fing.edu.svergara.xml2junit.XML2Junit parseYamlTestGenStrategyPath TestGenStrategy [artifactId=junit-circuit-breaker-resilience4j-notebook, groupId=uy.edu.fing.svergara, projectLocation=/Users/svergara/Desktop/tesis-maestria-eventb-compliance/test, types=[Type [name=CircuitBreaker, supertype=enum, values=[CLOSED, OPEN, HALF_OPEN]]], variables=[Variable [ignore=false, name=THRESHOLD, type=Integer], Variable [

##  STEP #4 (Resilience4j): Fill the Wrapper.java contents

In [9]:
cat test/junit-circuit-breaker-resilience4j-notebook/src/main/java/uy/edu/fing/svergara/Wrapper.java

package uy.edu.fing.svergara;

public class Wrapper {
	
	public void initialisation (Integer AMOUNT_TEST_REQUESTS, Integer THRESHOLD, Integer TIMEOUT_PERIOD, CircuitBreaker circuit_breaker, Integer consecutive_errors, Integer test_request_to_go, Integer time, Integer timestamp_cb_trips) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public Boolean isValid (CircuitBreaker circuit_breaker, Integer consecutive_errors, Integer test_request_to_go, Integer time, Integer timestamp_cb_trips) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public void request (Boolean microservice_response) {
		throw new RuntimeException("code not implemented yet");
	}
		
	public void clock () {
		throw new RuntimeException("code not implemented yet");
	}
		
}


##  STEP #5 (Resilience4j): Run circuit breaker implementation

Set JAVA_HOME to Java 16

In [123]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [124]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Package `circuit-breaker-resilience4j` project

In [10]:
mvn package -f circuit-breaker-resilience4j

[INFO] Scanning for projects...
[INFO] 
[INFO] --------< uy.edu.fing.uy.svergara:circuit-breaker-resilience4j >--------
[INFO] Building circuit-breaker-resilience4j 0.0.1-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:3.1.0:resources (default-resources) @ circuit-breaker-resilience4j ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 1 resource
[INFO] Copying 0 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.1:compile (default-compile) @ circuit-breaker-resilience4j ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- maven-resources-plugin:3.1.0:testResources (default-testResources) @ circuit-breaker-resilience4j ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Copying 0 resource
[INFO] 
[INFO] --- maven-compiler-plugin:3.8.1:testCompile (default-testCompile) @ circuit-breaker-resilience4j ---
[INFO] Nothing to compile - all c

The following will definitely not end running as it startup a tomcat server. Run it it in an independant terminal or run it with the detached flag (&) at the end. 

In [23]:
java -jar circuit-breaker-resilience4j/target/circuit-breaker-resilience4j-0.0.1-SNAPSHOT.jar &

[2] 752


##  STEP #6 (Resilience4j): Execute concrete test cases against implementation

In [131]:
export JAVA_HOME="/Library/Java/JavaVirtualMachines/temurin-16.jdk/Contents/Home"

In [132]:
java -version

openjdk version "16.0.2" 2021-07-20
OpenJDK Runtime Environment Temurin-16.0.2+7 (build 16.0.2+7)
OpenJDK 64-Bit Server VM Temurin-16.0.2+7 (build 16.0.2+7, mixed mode, sharing)


Run `junit-circuit-breaker-resilience4j` project (or the project generated in Step#3 and filled in Step#4)

In [25]:
mvn package -f test/junit-circuit-breaker-resilience4j

[INFO] Scanning for projects...
[INFO] 
[INFO] ------< uy.edu.fing.svergara:junit-circuit-breaker-resilience4j >-------
[INFO] Building junit-circuit-breaker-resilience4j 1.0-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ junit-circuit-breaker-resilience4j ---
[WARNING] Using platform encoding (UTF-8 actually) to copy filtered resources, i.e. build is platform dependent!
[INFO] skip non existing resourceDirectory /Users/svergara/Desktop/tesis-maestria-eventb-compliance/test/junit-circuit-breaker-resilience4j/src/main/resources
[INFO] 
[INFO] --- maven-compiler-plugin:3.1:compile (default-compile) @ junit-circuit-breaker-resilience4j ---
[INFO] Nothing to compile - all classes are up to date
[INFO] 
[INFO] --- maven-resources-plugin:2.6:testResources (default-testResources) @ junit-circuit-breaker-resilience4j ---
[WARNING] Using platform encoding (UTF-8 actually) to c